# Spectral Clustering

In [29]:
import numpy as np
import scipy as sp
import scipy.sparse.linalg
import scipy.cluster.vq

%matplotlib inline

In [3]:
A = np.loadtxt('../data/processed/usps.csv', delimiter=',')

In [38]:
inds = A[:, -1] < 2
X = A[inds, :-2]
Y = A[inds, -1]

In [39]:
n, d = X.shape
n, d

(1194, 255)

In [93]:
W = np.zeros((n, n))
for i in range(n):
    for j in range(i,n):
        val = np.e ** (-1 * np.linalg.norm(X[i] - X[j]) ** 2)
        W[i, j] = val
        W[j, i] = val

In [94]:
D = np.diag(W.sum(axis=1))

In [95]:
D_ = np.diag(1 / np.sqrt(W.sum(axis=1)))
L = np.identity(n) - D_.dot(W).dot(D_)

In [96]:
k = 2

In [97]:
V, Z = sp.sparse.linalg.eigsh(L, k=k+2)

In [98]:
Z.shape

(1194, 4)

In [99]:
Z_ = sp.cluster.vq.whiten(Z)
centroids, distortion = sp.cluster.vq.kmeans(Z_, k)

In [100]:
centroids

array([[-0.6597451 ,  0.87711529, -0.89311596, -1.01284616],
       [ 0.01585582, -0.02106969,  0.02145   ,  0.02424029]])

In [101]:
y_hat = np.zeros(n)
for i in range(n):
    d = np.array([np.linalg.norm(Z_[i] - centroids[c]) for c in range(k)])
    y_hat[i] = np.argmin(d)

In [102]:
accuracy = np.zeros(k)
for i in range(k):
    accuracy[i] = 1 - np.absolute(Y - ((y_hat + i) % k)).sum() / n * 100

In [103]:
errors.max()

55.69514237855946